In [1]:
import csv
import math
import itertools
from io import open
from conllu import parse_incr
import torch

In [2]:
# print the structure of the fine-tuning MLP for WSD
def print_fine_tuning_MLP(model, param):

	print('\n******************* fine-tuning MLP structure ***********************')

	print('Current Task: {}'.format(param))
	module_dict = model.layers[param]

	for module in module_dict:
		print(module)

	print('**********************************************************************')

In [3]:
# print the whole model 
def print_whole_model(model):

	print('\nAll parameters in the model:')
	for name, param in model.named_parameters():
		if param.requires_grad:
			print(name, param.size())

	print(model)

In [4]:
'''
return: 
all sentences
target word index
target word lemma
target word sense
all senses for each word 
from the EUD for train, test, and dev dataset
index provided by WSD dataset by White et. al.
'''
# get all the senses for each word from WSD dataset
def get_all_senses(wsd_data, train_data, test_data, dev_data):

    # all senses for each 
    all_senses = {}
    # for test purpose: only load specific amount of data
    for i in range(45):

        # get the original sentence from EUD
        sentence_id = wsd_data[i].get('Sentence.ID')
        
        # the index in EUD is 1-based!!!
        sentence_number = int(sentence_id.split(' ')[-1]) - 1
        # print('sentence id {} i {}'.format(sentence_id, i))
        word_index = int(wsd_data[i].get('Arg.Token')) - 1
        word_lemma = wsd_data[i].get('Arg.Lemma')
        word_sense = wsd_data[i].get('Synset')
        response = wsd_data[i].get('Sense.Response')

        # if the word already exits: add the new sense to the list
        # else: creata a new list for the word
        if word_lemma in all_senses.keys():
            if word_sense not in all_senses[word_lemma]:
                all_senses[word_lemma].append(word_sense)
        else:
            all_senses[word_lemma] = []
            all_senses[word_lemma].append(word_sense)            
            
    return all_senses

In [5]:
# parse the WSD dataset first
# and retrieve all sentences from the EUD

'''
Copyright@
White, A. S., D. Reisinger, K. Sakaguchi, T. Vieira, S. Zhang, R. Rudinger, K. Rawlins, & B. Van Durme. 2016. 
[Universal decompositional semantics on universal dependencies]
(http://aswhite.net/media/papers/white_universal_2016.pdf). 
To appear in *Proceedings of the Conference on Empirical Methods in Natural Language Processing 2016*.
'''

# parse the WSD dataset and construct X_Y tensors
def parse_data():

	# parse the EUD-EWT conllu files and retrieve the sentences
	train_file = open("data/UD_English-EWT/en_ewt-ud-train.conllu", "r", encoding="utf-8")
	train_data = list(parse_incr(train_file))
	print('Parsed {} training data from UD_English-EWT/en_ewt-ud-train.conllu.'.format(len(train_data)))

	test_file = open("data/UD_English-EWT/en_ewt-ud-test.conllu", "r", encoding="utf-8")
	test_data = list(parse_incr(test_file))
	print('Parsed {} testing data from UD_English-EWT/en_ewt-ud-test.conllu.'.format(len(test_data)))

	dev_file = open("data/UD_English-EWT/en_ewt-ud-dev.conllu", "r", encoding="utf-8")
	dev_data = list(parse_incr(dev_file))
	print('Parsed {} dev data from UD_English-EWT/en_ewt-ud-dev.conllu.'.format(len(dev_data)))

	# parse the WSD dataset
	wsd_data = []

	# read in tsv by White et. al., 2016
	with open('data/wsd/wsd_eng_ud1.2_10262016.tsv', mode = 'r') as wsd_file:

		tsv_reader = csv.DictReader(wsd_file, delimiter = '\t')      

		# store the data: ordered dict row
		for row in tsv_reader:                                

			# each data vector
			wsd_data.append(row)

		# make sure all data are parsed
		print('Parsed {} word sense data from White et. al., 2016.'.format(len(wsd_data)))

	return wsd_data, train_data, test_data, dev_data


In [6]:
# parse the data
wsd_data, train_data, test_data, dev_data = parse_data()

# return the raw sentences from the EUD for train, test, and dev
# test the first 20 sentences
all_senses = get_all_senses(wsd_data, train_data, test_data, dev_data)


Parsed 12543 training data from UD_English-EWT/en_ewt-ud-train.conllu.
Parsed 2077 testing data from UD_English-EWT/en_ewt-ud-test.conllu.
Parsed 2002 dev data from UD_English-EWT/en_ewt-ud-dev.conllu.
Parsed 439312 word sense data from White et. al., 2016.


In [7]:
'''
Construct the X and Y for train, dev, and test from White et. al., 2016
For each anonator and each word, on pair of data and label will be created
Warning: code here is hard to read LMAO
'''
def construct_X_Y(all_senses, train_data, dev_data, test_data):
    
    wsd_data = []
    
    with open('data/wsd/wsd_eng_ud1.2_10262016.tsv', mode = 'r') as wsd_file:
        
        tsv_reader = csv.DictReader(wsd_file, delimiter = '\t')
        
        # same annotator and same sentence number will generate on pair of X_Y
        # manually set the first sentence from White et. al., 2016
        current_annotator = '0'
        current_sentence_num = '1364'
        current_Y = [0 for _ in range(len(all_senses['spring']))]           
        sentence = train_data[1363]
        current_X = [word_dict.get('lemma') for word_dict in sentence]
        current_idx = 12
        
        # lists X and Y
        train_X, test_X, dev_X = ([] for i in range(3))
        train_Y, test_Y, dev_Y = ([] for i in range(3))
        train_word_idx, test_word_idx, dev_word_idx = ([] for i in range(3))

        for idx, row in enumerate(tsv_reader):
                        
            # training set; only test first 30 training sentences for now
            if idx < 45 and row['Split'] == 'train':
                
                # if still is the same annotatior, word index, target word
                # modify Y with the sense reponse
                # print('ca: {}, cs: {}'.format(current_annotator, current_sentence_num))
                if current_annotator == row['Annotator.ID'] and current_idx == int(row['Arg.Token']) - 1 and current_sentence_num == row['Sentence.ID'].split(' ')[-1]:
                    
                    sense_idx = all_senses[row['Arg.Lemma']].index(row['Synset'])
                    # print('h1: {}'.format(idx))
                    if row['Sense.Response'] == 'True':
                        current_Y[sense_idx] = 1
                    else:
                        current_Y[sense_idx] = 0
                
                # if switch annotator or target word
                # append the Y and X from the last annotator and word
                # start a new Y and X for the current annotator and target
                else:
                    # print('h2: {}'.format(idx))
                    train_X.append(current_X)
                    train_Y.append(current_Y)
                    train_word_idx.append(current_idx)
                    
                    current_annotator = row['Annotator.ID']
                    current_sentence_num = row['Sentence.ID'].split(' ')[-1]
                    
                    current_idx = int(row['Arg.Token']) - 1
                    current_Y = [0 for _ in range(len(all_senses[row['Arg.Lemma']]))]
                    sense_idx = all_senses[row['Arg.Lemma']].index(row['Synset'])
                    if row['Sense.Response'] == 'True':
                        current_Y[sense_idx] = 1
                    else:
                        current_Y[sense_idx] = 0
                    
                    sentence_id = row['Sentence.ID']
                    sentence_number = int(sentence_id.split(' ')[-1]) - 1
                    sentence = train_data[sentence_number]
                    current_X = [word_dict.get('lemma') for word_dict in sentence]
                    
            # testing set
            elif idx < 45 and row['Split'] == 'test':
                
                if current_annotator == row['Annotator.ID'] and current_idx == int(row['Arg.Token']) - 1 and current_sentence_num == row['Sentence.ID'].split(' ')[-1]:
                    
                    sense_idx = all_senses[row['Arg.Lemma']].index(row['Synset'])
                    if row['Sense.Response'] == 'True':
                        current_Y[sense_idx] = 1
                    else:
                        current_Y[sense_idx] = 0
                else:
                    test_X.append(current_X)
                    test_Y.append(current_Y)
                    test_word_idx.append(current_idx)

                    current_annotator = row['Annotator.ID']
                    current_sentence_num = row['Sentence.ID'].split(' ')[-1]

                    current_idx = int(row['Arg.Token']) - 1
                    current_Y = [0 for _ in range(len(all_senses[row['Arg.Lemma']]))]
                    sense_idx = all_senses[row['Arg.Lemma']].index(row['Synset'])
                    if row['Sense.Response'] == 'True':
                        current_Y[sense_idx] = 1
                    else:
                        current_Y[sense_idx] = 0
                    
                    sentence_id = row['Sentence.ID']
                    sentence_number = int(sentence_id.split(' ')[-1]) - 1
                    sentence = test_data[sentence_number]
                    current_X = [word_dict.get('lemma') for word_dict in sentence]
                    
            # dev set       
            elif idx < 45:
                if current_annotator == row['Annotator.ID'] and current_idx == int(row['Arg.Token']) - 1 and current_sentence_num == row['Sentence.ID'].split(' ')[-1]:
                    
                    sense_idx = all_senses[row['Arg.Lemma']].index(row['Synset'])
                    if row['Sense.Response'] == 'True':
                        current_Y[sense_idx] = 1
                    else:
                        current_Y[sense_idx] = 0
                else:
                    dev_X.append(current_X)
                    dev_Y.append(current_Y)
                    dev_word_idx.append(current_idx)

                    current_annotator = row['Annotator.ID']
                    current_sentence_num = row['Sentence.ID'].split(' ')[-1]

                    current_idx = int(row['Arg.Token']) - 1
                    current_Y = [0 for _ in range(len(all_senses[row['Arg.Lemma']]))]
                    sense_idx = all_senses[row['Arg.Lemma']].index(row['Synset'])
                    if row['Sense.Response'] == 'True':
                        current_Y[sense_idx] = 1
                    else:
                        current_Y[sense_idx] = 0
                    
                    sentence_id = row['Sentence.ID']
                    sentence_number = int(sentence_id.split(' ')[-1]) - 1
                    sentence = dev_data[sentence_number]
                    current_X = [word_dict.get('lemma') for word_dict in sentence]
        
        print('\n******************* Data Example ***********************')
        print('Sentence: {}'.format(train_X[0]))
        print('Annotator Response, i.e., true label: {}'.format(train_Y[0]))
        print('Target Word Index: {}'.format(train_word_idx[0]))
        print('All senses for the target word: {}'.format(all_senses[train_X[0][train_word_idx[0]]]))
        print('********************************************************')
        
        return train_X, test_X, dev_X, train_Y, test_Y, dev_Y, train_word_idx, test_word_idx, dev_word_idx
                    

In [8]:
# print(all_senses)
train_X, test_X, dev_X, train_Y, test_Y, dev_Y, train_word_idx, test_word_idx, dev_word_idx = construct_X_Y(all_senses, train_data, dev_data, test_data)
# print('train_X: {}'.format(train_X))
# print('train_Y: {}'.format(train_Y))


******************* Data Example ***********************
Sentence: ['on', 'August', '9', ',', '2004', ',', 'it', 'be', 'announce', 'that', 'in', 'the', 'spring', 'of', '2001', ',', 'a', 'man', 'name', 'El', '-', 'Shukrijumah', ',', 'also', 'know', 'as', 'Jafar', 'the', 'Pilot', ',', 'who', 'be', 'part', 'of', 'a', '"', 'second', 'wave', ',', '"', 'have', 'be', 'case', 'New', 'York', 'City', 'helicopter', '.']
Annotator Response, i.e., true label: [1, 0, 0, 0, 0, 0]
Target Word Index: 12
All senses for the target word: ['spring.n.01', 'spring.n.02', 'spring.n.03', 'spring.n.04', 'give.n.01', 'leap.n.01']
********************************************************


In [9]:
from allennlp.commands.elmo import ElmoEmbedder
from model import *

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [10]:
# ELMo setup
# ELMo is tuned to lower dimension (256) by MLP in Model
elmo = ElmoEmbedder()
model = Model(elmo_class = elmo, all_senses = all_senses)

# print the model 
print_whole_model(model)

# MLP illustration
print_fine_tuning_MLP(model, 'WSD')


All parameters in the model:
layers.WSD.0.weight torch.Size([300, 512])
layers.WSD.0.bias torch.Size([300])
dimension_reduction_MLP.weight torch.Size([256, 3072])
dimension_reduction_MLP.bias torch.Size([256])
wsd_lstm.weight_ih_l0 torch.Size([1024, 256])
wsd_lstm.weight_hh_l0 torch.Size([1024, 256])
wsd_lstm.bias_ih_l0 torch.Size([1024])
wsd_lstm.bias_hh_l0 torch.Size([1024])
wsd_lstm.weight_ih_l0_reverse torch.Size([1024, 256])
wsd_lstm.weight_hh_l0_reverse torch.Size([1024, 256])
wsd_lstm.bias_ih_l0_reverse torch.Size([1024])
wsd_lstm.bias_hh_l0_reverse torch.Size([1024])
wsd_lstm.weight_ih_l1 torch.Size([1024, 512])
wsd_lstm.weight_hh_l1 torch.Size([1024, 256])
wsd_lstm.bias_ih_l1 torch.Size([1024])
wsd_lstm.bias_hh_l1 torch.Size([1024])
wsd_lstm.weight_ih_l1_reverse torch.Size([1024, 512])
wsd_lstm.weight_hh_l1_reverse torch.Size([1024, 256])
wsd_lstm.bias_ih_l1_reverse torch.Size([1024])
wsd_lstm.bias_hh_l1_reverse torch.Size([1024])
output_layers.spring.weight torch.Size([6, 30

In [11]:
# forward propagation
# ELMo (1024) -> dimension reduction (256) -> bi-LSTM (512) -> fine-tuning MLP (10)
results_45 = model.forward(train_X, train_word_idx)


Original ELMo embeddings size: torch.Size([8, 3, 48, 1024]), mask: torch.Size([8, 48])
Embedding size after dimension reduction: torch.Size([48, 8, 256]), mask: torch.Size([48, 8, 256])

Embedding size after bi-LSTM: torch.Size([48, 8, 512])

Word lemma: spring
Embedding vector (distributions over all its senses): tensor([0.4938, 0.5077, 0.4835, 0.5154, 0.5064, 0.5165],
       grad_fn=<SigmoidBackward>)
All its senses: ['spring.n.01', 'spring.n.02', 'spring.n.03', 'spring.n.04', 'give.n.01', 'leap.n.01']

Word lemma: spring
Embedding vector (distributions over all its senses): tensor([0.4938, 0.5077, 0.4835, 0.5154, 0.5064, 0.5165],
       grad_fn=<SigmoidBackward>)
All its senses: ['spring.n.01', 'spring.n.02', 'spring.n.03', 'spring.n.04', 'give.n.01', 'leap.n.01']

Word lemma: ambition
Embedding vector (distributions over all its senses): tensor([0.5000, 0.4949], grad_fn=<SigmoidBackward>)
All its senses: ['ambition.n.01', 'ambition.n.02']

Word lemma: management
Embedding vector (